<a href="https://colab.research.google.com/github/elichen/karpathyGPT/blob/main/SouthPark_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import csv

In [ ]:
block_size = 256
batch_size = 64
n_layer = 4
n_head = 4
n_embed = 256
n_ffwd = n_embed * 4
dropout = 0.2
learning_rate = 3e-4
max_inters = 5000
eval_iters = 200
eval_interval = 500
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
if not os.path.exists("All-seasons.csv"):
  !wget -q "https://raw.githubusercontent.com/BobAdamsEE/SouthParkData/master/All-seasons.csv"
with open('All-seasons.csv', 'r') as csv_file, open('output.txt', 'w') as out_file:
    for row in csv.DictReader(csv_file):
        out_file.write(f"{row['Character']}: {row['Line'].strip()}\n")
with open('output.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size, "".join(chars)

In [ ]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [ ]:
data = torch.tensor(encode(text))
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x.to(device), y.to(device)

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, head_dim):
        super().__init__()
        self.query = nn.Linear(n_embed, head_dim)
        self.key = nn.Linear(n_embed, head_dim)
        self.value = nn.Linear(n_embed, head_dim)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        wei = q @ q.transpose(-2, -1) * k.shape[1] ** -0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        output = wei @ v
        return output

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_dim):
        super().__init__()
        self.heads = nn.ModuleList([AttentionHead(head_dim) for _ in range(num_heads)])
        self.linear = nn.Linear(num_heads * head_dim, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.linear(out)
        out = self.dropout(out)
        return out

In [ ]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
          nn.Linear(n_embed, n_ffwd),
          nn.ReLU(),
          nn.Linear(n_ffwd, n_embed),
          nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    def __init__(self, num_heads):
        super().__init__()
        self.sa_heads = MultiHeadAttention(num_heads, n_embed//num_heads)
        self.ffwd = FeedForward()
        self.norm1 = nn.LayerNorm(n_embed)
        self.norm2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa_heads(self.norm1(x))
        x = x + self.ffwd(self.norm2(x))
        return x

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embeddings = nn.Embedding(vocab_size, n_embed)
        self.position_embeddings = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_head) for _ in range(n_layer)])
        self.norm = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, inputs):
        B,T = inputs.shape
        tok_emb = self.token_embeddings(inputs)
        pos_emb = self.position_embeddings(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.norm(x)
        return self.lm_head(x)

    def calculate_loss(self, inputs, targets):
        logits = self(inputs)
        return F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))

    def generate(self, input_batch, max_new_tokens):
        for _ in range(max_new_tokens):
            logits = self(input_batch[:,-block_size:])
            last_logits = logits[:, -1, :]
            probabilities = F.softmax(last_logits, dim=-1)
            next_tokens = torch.multinomial(probabilities, 1).squeeze(1)
            input_batch = torch.cat((input_batch, next_tokens.unsqueeze(1)), dim=1)
        return input_batch

In [ ]:
@torch.no_grad()
def estimate_loss(model):
  out = {}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      loss = model.calculate_loss(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [ ]:
weights_file = 'southpark.pt'

In [ ]:
# ![[ -f {weights_file} ]] || wget -O {weights_file} https://github.com/elichen/karpathyGPT/raw/main/{weights_file}

In [ ]:
model = LanguageModel().to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')
if not os.path.exists(weights_file):
  xb,yb = get_batch("train")
  optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

  for steps in range(max_inters):
    if steps % eval_interval == 0:
      losses = estimate_loss(model)
      print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb,yb = get_batch("train")
    loss = model.calculate_loss(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  torch.save(model.state_dict(), weights_file)
else:
  model.load_state_dict(torch.load(weights_file, map_location=torch.device(device)))

3.299473 M parameters


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=1000)[0].tolist()))